Analysis of Gridded Ensemble Precipitation and Temperature Estimates over the Contiguous United States
====

For this example, we'll work with 100 member ensemble of precipitation and temperature data. 

Link to dataset: https://www.earthsystemgrid.org/dataset/gridded_precip_and_temp.html

In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

### Connect to Dask Distributed Cluster

In [ ]:
from dask.distributed import Client
from dask_kubernetes import KubeCluster
cluster = KubeCluster()
# cluster.adapt(maximum=20)
cluster.scale(2)
cluster

In [ ]:
client = Client(cluster)
client

### Open Dataset

We try storing a traditional atmospheric dataset on the cloud with two approaches

1.  Storing the data in an easier-to-read but new format, [Zarr](http://zarr.readthedocs.io/en/stable/)
1.  Loading an [Intake](https://intake.readthedocs.io/en/latest/) catalog file from GCS, specifying the location of the zarr data.

The dataset has dimensions of time, latitude, longitude, and ensmemble member. Each format is self-describing.

In [ ]:
### Load with intake catalog service
import intake
cat = intake.Catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")
ds = cat["gmet_v1"].to_dask()
ds

In [ ]:
# Print dataset
ds

### Figure: Elevation and domain mask

A quick plot of the mask to give us an idea of our spatial domain

In [ ]:
elevation = ds['elevation']
elevation = elevation.where(elevation > 0).load()
elevation.plot(figsize=(10, 6))
plt.title('Domain Elevation')

### Quantify the ensemble uncertainty for a single day

This dataset provides 100 equally likely realizations of the temperature/precipitation that could have occured, given the station-observed weather. We can quantify the uncertaintly that comes from observation and gridding errors like this:

In [ ]:
temp = ds['t_mean'].sel(time='1984-07-31')
temp_ens_mean = temp.mean('ensemble')
temp_errors = temp - temp_ens_mean
temp_std_errors = temp_errors.std('ensemble')

In [ ]:
temp_std_errors.plot(robust=True)

As we can see, remote and topographically complex areas tend to have larger uncertainties in this dataset.

### Intra-ensemble range

We calculate the intra-ensemble range for all the mean daily temperature in this dataset.  This gives us a sense of uncertainty.

In [ ]:
temp_mean = ds['t_mean'].mean(dim='time')
spread = (temp_mean.max(dim='ensemble')
        - temp_mean.min(dim='ensemble'))
spread

### Calling compute
The expressions above didn't actually compute anything. They just build the task graph. To do the computations, we call the `compute` or `persist` methods:

In [ ]:
spread = spread.persist(retries=2)
spread

#### Figure: Intra-ensemble range


In [ ]:
spread.attrs['units'] = 'degC'
spread.plot(robust=True, figsize=(10, 6))
plt.title('Intra-ensemble range in mean annual temperature')

### Average seasonal snowfall

We can compute a crude estimate of average seasonal snowfall using the temperature and precipitation variables in our dataset. Here, we'll look at the first 4 ensemble members and make some maps of the seasonal total snowfall in each ensemble member.

In [ ]:
da_snow = ds['pcp'].where(ds['t_mean'] < 0.).resample(time='QS-Mar').sum('time')
seasonal_snow = da_snow.isel(ensemble=slice(0, 4)).groupby('time.season').mean('time').persist()

In [ ]:
# properly sort the seasons
seasonal_snow = seasonal_snow.sel(season=['DJF', 'MAM','JJA', 'SON'])
seasonal_snow.attrs['units'] = 'mm/season'
seasonal_snow

#### Figure: Average seasonal snowfall totals 

In [ ]:
seasonal_snow.plot.pcolormesh(col='season', row='ensemble', cmap='Blues', robust=True)

### Extract a time series of annual maximum precipitation events over a region

In the previous two examples, we've mostly reduced the time and/or ensemble dimension. Here, we'll do a reduction operation on the spatial dimension to look at a time series of extreme precipitation events near Austin, TX (30.2672° N, 97.7431° W).

In [ ]:
buf = 0.25  # look at Austin +/- 0.25 deg

ds_tx = ds.sel(lon=slice(-97.7431-buf, -97.7431+buf), lat=slice(30.2672-buf, 30.2672+buf))

In [ ]:
pcp_ann_max = ds_tx['pcp'].resample(time='AS').max('time')

In [ ]:
pcp_ann_max_ts = pcp_ann_max.max(('lat', 'lon')).persist()
pcp_ann_max_ts

#### Figure: Timeseries of maximum precipitation near Austin, Tx.

In [ ]:
ax = pcp_ann_max_ts.transpose().to_pandas().plot(title='Maximum precipitation near Austin, Tx', legend=False)